In [ ]:
pip install PyPDF2

In [ ]:
import PyPDF2
import json

In [ ]:
def extract_text_from_pdf(pdf_path):
    pdf_document = open(pdf_path,'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_document)
    num_pages = len(pdf_reader.pages)
    
    text = ""
    for page in pdf_reader.pages:
            # page = pdf_reader.getPage(page_num)
            text += page.extract_text()
    
    return text

In [ ]:
pdf_path = "Alaska Airlines and Horizon Air SMS Manual.pdf"
extracted_text = extract_text_from_pdf(pdf_path)

if extracted_text:
    print("extracted_text")
else:
    print("Unable to extract text from the PDF")

In [ ]:
extracted_text

In [ ]:
#provided pdf text
pdf_text = extracted_text

# Define Section titles
section_titles = [
"Introduction",
"Revision Record",
"Purpose",
"Scope"
]

#intialize a dictionary to store the sections
sections = {}
 
#Split the PDF text by section titles
for i in range(len(section_titles)):
    title = section_titles[i]

    if i < len(section_titles) -1:
        # print (pdf_text.split(title,1)[1].split(section_titles[i+1],1)[0].strip())
        section_text = pdf_text.split(title,1)[1].split(section_titles[i+1],1)[0].strip()
        sections[title] = section_text
    else:
        selection_text = pdf_text.split(title,1)[1].strip()
        sections[title] = section_text
    
    json_data = json.dumps(sections,indent=4)

final = json.loads(json_data)
print(final['Scope'])

In [ ]:
https://testingchat.openai.azure.com/

In [ ]:
# pip install langchain
#!pip install openai
#!pip install pypdf
#!pip install tiktoken
#!pip install faiss-cpu
# Install Azure Storage Blob package
#!pip install azure-storage-blob
!pip install AzureOpenAI

In [106]:
#import openai
#import streamlit as st
#import langchain
from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain.vectorstores import FAISS
import tempfile
import time
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
#from typing_extensions import Concatenate
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain

In [107]:
question = 'safety Actions'
pdf_file_name = 'Alaska Airlines and Horizon Air SMS Manual.pdf'
api_key = 'key'

llm = OpenAI(openai_api_key = api_key, temperature=0)
# We need to split the text using Character Text Split such that it should not increase token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)

In [108]:
def pdf_text_extraction(pdf_file_path):
    if pdf_file_path is not None:
        # Open the PDF file and read its contents
        with open(pdf_file_path, 'rb') as pdf_file:
            # Create a temporary file to write the PDF contents
            with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
                tmp_file.write(pdf_file.read())
                pdf_path = tmp_file.name

        # Now you can use the PyPDFLoader on the temporary PDF file
        loader = PyPDFLoader(pdf_path)
        pages_split = loader.load_and_split()
    return pages_split

pages = pdf_text_extraction(pdf_file_name)   

FileNotFoundError: [Errno 2] No such file or directory: 'Alaska Airlines and Horizon Air SMS Manual.pdf'

In [ ]:
def doc_similarity_search(pages, question):
    combined_content = ''.join([p.page_content for p in pages])
    texts = text_splitter.split_text(combined_content)
    embedding = OpenAIEmbeddings(openai_api_key = api_key)
    document_search = FAISS.from_texts(texts, embedding)
    #print('docs:',document_search)
    chain = load_qa_chain(llm, chain_type="stuff")
    docs = document_search.similarity_search(question)
    return docs

documents = doc_similarity_search(pages, question)

In [109]:
from azure.storage.blob import BlobServiceClient

# Replace with your actual connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"


# Create a BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Replace with your container name
container_name = "safety"

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

# List blobs in the container
blobs = container_client.list_blobs()
for blob in blobs:
    if blob.name.endswith(".pdf"):  # Check if the blob is a PDF file
        blob_client = container_client.get_blob_client(blob)
        pdf_data = blob_client.download_blob().readall()  
        pages = text_extraction(pdf_file_name)   

NameError: name 'text_extraction' is not defined

In [ ]:
for p in pages:
    print(p)
    print('\n')

In [ ]:
documents = doc_similarity_search(pages, question)
for doc in documents:
    print(doc.page_content)
    print('\n')

In [ ]:
--send data to cognitive search

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

# Replace with your actual values
service_name = "your-search-service-name"
index_name = "your-index-name"
api_version = "2020-06-30"
api_key = "your-api-key"

# Create a SearchClient
endpoint = f"https://{service_name}.search.windows.net"
credential = AzureKeyCredential(api_key)
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

# Define the data you want to upload
data = [
    {"id": "1", "title": "Document 1", "content": "This is the content of document 1."},
    {"id": "2", "title": "Document 2", "content": "This is the content of document 2."}
]

# Upload data to the index
search_client.upload_documents(documents=data)


In [105]:
#from azure_openai import AzureOpenAI, CharacterTextSplitter, PyPDFLoader, OpenAIEmbeddings, FAISS
from azure.openai import OpenAIClient  #, CharacterTextSplitter, PyPDFLoader, OpenAIEmbeddings, FAISS
import tempfile
from azure.storage.blob import BlobServiceClient

# Replace with your Azure OpenAI API key and endpoint
azure_openai_api_key = 'e8143eabb02541259054426630db12a7'
azure_openai_endpoint = 'https://testingchat.openai.azure.com/'

question = 'safety Actions'
pdf_file_name = 'Alaska Airlines and Horizon Air SMS Manual.pdf'

# Initialize AzureOpenAI with your API key
azure_openai = AzureOpenAI(api_key=azure_openai_api_key, endpoint=azure_openai_endpoint, temperature=0)

# We need to split the text using Character Text Split such that it should not increase token size
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)

def pdf_text_extraction(pdf_file_path):
    if pdf_file_path is not None:
        # Open the PDF file and read its contents
        with open(pdf_file_path, 'rb') as pdf_file:
            # Create a temporary file to write the PDF contents
            with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
                tmp_file.write(pdf_file.read())
                pdf_path = tmp_file.name

        # Now you can use the PyPDFLoader on the temporary PDF file
        loader = PyPDFLoader(pdf_path)
        pages_split = loader.load_and_split()
    return pages_split

pages = pdf_text_extraction(pdf_file_name)

def doc_similarity_search(pages, question):
    combined_content = ''.join([p.page_content for p in pages])
    texts = text_splitter.split_text(combined_content)
    embedding = OpenAIEmbeddings(api_key=azure_openai_api_key, endpoint=azure_openai_endpoint)
    document_search = FAISS.from_texts(texts, embedding)
    chain = azure_openai.load_qa_chain(chain_type="stuff")
    docs = document_search.similarity_search(question)
    return docs

documents = doc_similarity_search(pages, question)

# Replace with your actual connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"

# Create a BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Replace with your container name
container_name = "safety"

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

# List blobs in the container
blobs = container_client.list_blobs()
for blob in blobs:
    if blob.name.endswith(".pdf"):  # Check if the blob is a PDF file
        blob_client = container_client.get_blob_client(blob)
        pdf_data = blob_client.download_blob().readall()
        pages = pdf_text_extraction(pdf_file_name)

documents = doc_similarity_search(pages, question)
for doc in documents:
    print(doc.page_content)
    print('\n')


ModuleNotFoundError: No module named 'azure.openai'

In [110]:
question = 'safety Actions'
pdf_file_name = 'Alaska Airlines and Horizon Air SMS Manual.pdf'
api_key = 'key'

llm = OpenAI(openai_api_key = api_key, temperature=0)
# We need to split the text using Character Text Split such that it should not increase token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)

def pdf_text_extraction(pdf_file_path):
    if pdf_file_path is not None:
        # Open the PDF file and read its contents
        with open(pdf_file_path, 'rb') as pdf_file:
            # Create a temporary file to write the PDF contents
            with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
                tmp_file.write(pdf_file.read())
                pdf_path = tmp_file.name

        # Now you can use the PyPDFLoader on the temporary PDF file
        loader = PyPDFLoader(pdf_path)
        pages_split = loader.load_and_split()
    return pages_split

pages = pdf_text_extraction(pdf_file_name)   


def doc_similarity_search(pages, question):
    combined_content = ''.join([p.page_content for p in pages])
    texts = text_splitter.split_text(combined_content)
    embedding = OpenAIEmbeddings(openai_api_key = api_key)
    document_search = FAISS.from_texts(texts, embedding)
    #print('docs:',document_search)
    chain = load_qa_chain(llm, chain_type="stuff")
    docs = document_search.similarity_search(question)
    return docs

documents = doc_similarity_search(pages, question)


from azure.storage.blob import BlobServiceClient

# Replace with your actual connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"


# Create a BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Replace with your container name
container_name = "safety"

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

# List blobs in the container
blobs = container_client.list_blobs()
for blob in blobs:
    if blob.name.endswith(".pdf"):  # Check if the blob is a PDF file
        blob_client = container_client.get_blob_client(blob)
        pdf_data = blob_client.download_blob().readall()  
        pages = text_extraction(pdf_file_name)   
        
documents = doc_similarity_search(pages, question)
for doc in documents:
    print(doc.page_content)
    print('\n')


FileNotFoundError: [Errno 2] No such file or directory: 'Alaska Airlines and Horizon Air SMS Manual.pdf'

In [ ]:
https://medium.com/mlearning-ai/using-langchain-vector-store-using-azure-cognitive-search-4ab89b626b92
    
https://www.youtube.com/watch?v=FW_d5F1v54U

In [7]:
!pip install azure-search-documents==11.4.0b8
!pip install azure-identity
!pip install openai
!pip install langchain

In [26]:
import openai
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

In [27]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://testingchat.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "e8143eabb02541259054426630db12a7"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
model: str = "text-embedding-ada-002"
    
#e8143eabb02541259054426630db12a7    

In [13]:
#vector_store_address: str = "https://genaisafety.search.windows.net"
#vector_store_password: str = "iDXpfwNsnwyN88dYUlj3c0TaXP76z1DlWlSxZ9QmPxAzSeAu7ppU"

In [36]:
AZURE_SEARCH_SERVICE_ENDPOINT = "https://genaisafety.search.windows.net"
AZURE_SEARCH_ADMIN_KEY = "jA50VnyT0OAeuHpY0RxJGz7z9kfrmdj7YF5PHYiPXjAzSeBS6hJg"
AZURE_SEARCH_INDEX_NAME = "test-event-index"

In [100]:
#embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment="textembeddingada", model = "text-embedding-ada-002", chunk_size=1)
embeddings = OpenAIEmbeddings(deployment = model,chunk_size=1) 
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    azure_search_key=AZURE_SEARCH_ADMIN_KEY,
    index_name = AZURE_SEARCH_INDEX_NAME,
    embedding_function=embeddings.embed_query,
)

In [41]:
def pdf_text_extraction(pdf_file_path):
    if pdf_file_path is not None:
        # Open the PDF file and read its contents
        with open(pdf_file_path, 'rb') as pdf_file:
            # Create a temporary file to write the PDF contents
            with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
                tmp_file.write(pdf_file.read())
                pdf_path = tmp_file.name

        # Now you can use the PyPDFLoader on the temporary PDF file
        loader = PyPDFLoader(pdf_path)
        pages_split = loader.load_and_split()
    return pages_split

In [53]:
import tempfile
from langchain.document_loaders import PyPDFLoader
pdf_file_name = "Alaska Airlines and Horizon Air SMS Manual.pdf"

from azure.storage.blob import BlobServiceClient

# Replace with your actual connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"
# Create a BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Replace with your container name
container_name = "safety"

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

# List blobs in the container
blobs = container_client.list_blobs()
for blob in blobs:
    if blob.name.endswith(".pdf"):  # Check if the blob is a PDF file
        blob_client = container_client.get_blob_client(blob)
        pdf_files = blob_client.download_blob().readall()  
        #pages = pdf_text_extraction(pdf_file_name)
        loader = PyPDFLoader(pdf_file_name)

In [73]:
blobs = container_client.list_blobs()
for blob in blobs:
    print(blob.name)

manuals/AAG_Safety_Manual.pdf
manuals/Alaska Airlines and Horizon Air SMS Manual.pdf
sample-etq-data/EventData.csv


In [102]:
loader = PyPDFLoader("Alaska Airlines and Horizon Air SMS Manual.pdf")
pages = loader.load_and_split()

In [103]:
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs)

HttpResponseError: () The request is invalid. Details: parameters : The property 'id' does not exist on type 'search.documentFields'. Make sure to only use property names that are defined by the type.
Code: 
Message: The request is invalid. Details: parameters : The property 'id' does not exist on type 'search.documentFields'. Make sure to only use property names that are defined by the type.

In [ ]:
# Perform a similarity search
docs = vector_store.similarity_search(
    query="safety Actions",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)

In [104]:
docs

[Document(page_content='Uncontrolled copy when downloaded or printed from an electronic document.\nSafety Management System \n(SMS) Manual\nAlaska Airlines, Inc. / Horizon Air\n(Alaska Air Group)\nRevision 18 6/16/2023', metadata={'source': 'Alaska Airlines and Horizon Air SMS Manual.pdf', 'page': 0}),
 Document(page_content='Safety Management System Manual\nUncontrolled copy when downloaded or printed from an electronic document.\nIntentionally Left Blank', metadata={'source': 'Alaska Airlines and Horizon Air SMS Manual.pdf', 'page': 1}),
 Document(page_content='i of vi\nRevision HighlightsSafety Management System Manual\nUncontrolled copy when downloaded or printed from an electronic document.\nRevision Highlights\nChanges inserted with this revision are listed in the table below.\nTable 1-1: \nChapter Description\nAll Changed references of “risk matrix” to “SMS Matrix”\nCorrected minor spelling and grammatical errors\nFixed minor innacuracies\nChanged instances of “corrective action